In [8]:
# Imports
import pandas as pd
import numpy as np
import math
import matplotlib.pyplot as plt
import seaborn as sb
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import ConfusionMatrixDisplay, accuracy_score, classification_report, confusion_matrix
from sklearn.tree import DecisionTreeClassifier

df_coaches = pd.read_csv('dataset/processed/coaches_processed.csv')

stats_df_coaches = df_coaches.drop("stint", axis=1)

stats_df_coaches["Win Rate"] = np.where(
    (stats_df_coaches['post_wins'] == 0) & (stats_df_coaches['post_losses'] == 0),
    stats_df_coaches['won'] / (stats_df_coaches['won'] + stats_df_coaches['lost']),
    ((stats_df_coaches['won'] + stats_df_coaches['post_wins']) / (stats_df_coaches['won'] + stats_df_coaches['lost'] + stats_df_coaches['post_wins'] + stats_df_coaches['post_losses']))
)

#Estreante ou não
stats_df_coaches['Cumulative Win Rate'] = stats_df_coaches.groupby('coachID')['Win Rate'].expanding().mean().reset_index(level=0, drop=True).shift(1).fillna(0)

stats_df_coaches.drop(columns=['won', 'lost', 'post_wins', 'post_losses', 'Win Rate'], inplace=True) 

df_teams = pd.read_csv('dataset/processed/teams_test.csv')

df_teams = pd.merge(df_teams, stats_df_coaches, on=['tmID', 'year'], how='left')

df_teams.to_csv('dataset/processed/teams_test.csv', index=False)

stats_df_coaches.head()
#stats_df_players.dropna(axis=0, how='any', inplace=True)
#stats_df_players.isna().sum()

#gajos com 0 minutos
#df_players.loc[df_players['minutes'] <= 0, ['minutes', 'PostMinutes']]



,coachID,year,tmID,Cumulative Win Rate
0,adamsmi01w,5,WAS,0.500000
1,adubari99w,1,NYL,0.486486
2,adubari99w,2,NYL,0.615385
3,adubari99w,3,NYL,0.623482
4,adubari99w,4,NYL,0.598988
